# MCP Gateway & DAG Workflow Visualization

Demonstrates DAG execution and graph visualization.

In [ ]:
import { WorkflowDAGStore } from "jsr:@casys/mcp-gateway";
import { renderToSVG } from "jsr:@casys/mcp-gateway";

console.log("✅ MCP Gateway loaded");

## Create a Simple DAG Workflow

In [ ]:
const workflow = {
  tasks: {
    fetchData: {
      tool: "sandbox_execute",
      params: { code: "return { data: [1,2,3,4,5] }" },
      dependencies: []
    },
    processData: {
      tool: "sandbox_execute",
      params: { code: "return context.data.map(x => x * 2)" },
      dependencies: ["fetchData"]
    },
    aggregate: {
      tool: "sandbox_execute",
      params: { code: "return context.reduce((a,b) => a+b, 0)" },
      dependencies: ["processData"]
    }
  }
};

console.log("Workflow defined:", Object.keys(workflow.tasks));

## Visualize DAG with Graphviz

Generate SVG visualization of the workflow graph:

In [ ]:
// Generate DOT notation
const dotGraph = `
digraph Workflow {
  rankdir=TB;
  node [shape=box, style=filled, fillcolor=lightblue];
  
  fetchData [label="fetchData\nGet data"];
  processData [label="processData\nTransform"];
  aggregate [label="aggregate\nSum results"];
  
  fetchData -> processData;
  processData -> aggregate;
}
`;

// Display using HTML
const svg = `<svg width="400" height="300" xmlns="http://www.w3.org/2000/svg">
  <rect x="50" y="20" width="120" height="50" fill="lightblue" stroke="black"/>
  <text x="110" y="50" text-anchor="middle">fetchData</text>
  
  <rect x="50" y="120" width="120" height="50" fill="lightblue" stroke="black"/>
  <text x="110" y="150" text-anchor="middle">processData</text>
  
  <rect x="50" y="220" width="120" height="50" fill="lightblue" stroke="black"/>
  <text x="110" y="250" text-anchor="middle">aggregate</text>
  
  <line x1="110" y1="70" x2="110" y2="120" stroke="black" marker-end="url(#arrowhead)"/>
  <line x1="110" y1="170" x2="110" y2="220" stroke="black" marker-end="url(#arrowhead)"/>
  
  <defs>
    <marker id="arrowhead" markerWidth="10" markerHeight="10" refX="9" refY="3" orient="auto">
      <polygon points="0 0, 10 3, 0 6" fill="black" />
    </marker>
  </defs>
</svg>`;

console.log("📊 Workflow Graph:");
console.log(svg);

## Execute DAG Workflow

Run the workflow and track execution:

In [ ]:
import { DenoSandboxExecutor } from "jsr:@casys/mcp-gateway";

const sandbox = new DenoSandboxExecutor({ timeout: 5000 });

// Simulate DAG execution
const results = {};

const task1 = await sandbox.execute("return { data: [1,2,3,4,5] }");
results.fetchData = task1.result;
console.log("✅ fetchData:", results.fetchData);

const task2 = await sandbox.execute(
  "return context.data.map(x => x * 2)",
  results.fetchData
);
results.processData = task2.result;
console.log("✅ processData:", results.processData);

const task3 = await sandbox.execute(
  "return context.reduce((a,b) => a+b, 0)",
  results.processData
);
results.aggregate = task3.result;
console.log("✅ aggregate:", results.aggregate);

console.log("\n🎉 Workflow completed! Final result:", results.aggregate);